# Matrix Inversion as a QUBO problem
This notebook is a reference implementation for the paper "Floating-Point Calculations on a Quantum Annealer: Division and Matrix Inversion" by M. Rogers and R. Singleton, 2020 [1]. The paper describes an approach to floating-point calculations on a quantum annealer. Specifically, they introduce a binary representation of the floating point variables. This representation is discrete, as they demonstrate their technique for 4 bit and 8 bit accuracy. With this measure they explain and derive the inversion of a matrix as a QUBO problem, which is suitable to run on a DWave quantum annealer. 

[1] Rogers, Michael L., and Robert L. Singleton Jr. "Floating-point calculations on a quantum annealer: Division and matrix inversion." Frontiers in Physics 8 (2020): 265.

In [29]:
import numpy as np
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler, AutoEmbeddingComposite, LazyEmbeddingComposite, CutOffComposite, PolyCutOffComposite, LazyFixedEmbeddingComposite, FixedEmbeddingComposite, TilingComposite, VirtualGraphComposite, ReverseBatchStatesComposite, ReverseAdvanceComposite

The main challenge in formulating problems for quantum annealers is the description as a binary problem. The authors formulate their problem as a QUBO problem. 

Scalar notation of a QUBO problem: 

\begin{equation} E_{qubo}(a_i, b_{i,j}; q_i) = \sum_{i} a_i q_i + \sum_{i < j} b_{i, j} q_i q_j 
\end{equation} 

Matrix (NxN) inversion is defined as: 

\begin{equation} M \cdot x = y \rightarrow x=M^{-1} \cdot y 
\end{equation} 

Formulate matrix inversion as a quadratic minimization problem with its minimum being the matrix inverse:

\begin{equation} H(x) = (Mx-y)^2 = \sum_{ijk=1}^{N} M_{ki} M_{kj} x^i x^j - 2 \sum_{ij=1}^{N} y_j M_{ji} x^i + \| y \|^2
\end{equation} 


We can obtain a floating point representation of each component of x by expanding in powers of 2 multiplied by boolean-valued variables $q_r^i \in \{0,1\}$ : 


\begin{equation} \chi^i = \sum_{r=0}^{R-1} 2^{-r} q_{r}^{i}
\end{equation} 

\begin{equation} x^i = 2\chi^i -1
\end{equation} 

In [3]:
def compute_chi(q, binary_range):
    # q: Value of binary qubits e.g. q = [0, 1, 0, 1]
    # Binary range is R: amount of binary variables to represent floating precision e.g. 4
    chi = 0
    for r in range(0, binary_range):
        chi += 2**(-r) * q[r]
    return chi

def compute_x(chi):
    # This is just an intermediate step...
    return 2*chi - 1

Let's confirm this with the smallest (-1.0) and largest element (2.75) in the range:

In [4]:
# Smallest element
q_start = np.array([0, 0, 0, 0])
# Largest element
q_end = np.array([1, 1, 1, 1])
# Range is length of bidary number
binary_range = q_start.shape[0]
# Compute value...
x_start = compute_x(compute_chi(q_start, binary_range))
x_end = compute_x(compute_chi(q_end, binary_range))
print(x_start)
print(x_end)

-1.0
2.75


So now our problem can be formulated by expressing x as a function $q_r^i$:

\begin{equation} H[q] = \sum_{i=1}^{N} \sum_{r=0}^{R-1} a_r^i q_r^i + \sum_{i=1}^{N} \sum_{i \neq j}^{N} \sum_{r=0}^{R-1} \sum_{s=0}^{R-1} b_{rs}^{ij} q_r^i q_s^i
\end{equation} 

With this we can formulate our QUBO coefficients. For a detailed derivation please refer to the paper:

\begin{equation} a_r^i = 4 \cdot 2^{-r} \sum_{k} M_{ki} \{ 2^{-r} M_{ki} - (y_k + \sum_{j} M_{kj}) \}
\end{equation} 

\begin{equation} b_{rs}^{ij} = 4 \cdot 2^{-(r+s)} \sum_{k} M_{ki} M_{kj} 
\end{equation} 

In the DWave system QUBO coefficients are addressed with a 1-D index, therefore we have to address the components with a 1-dimensional linear index. So the 2D-indices i (0, N-1) and r (0, R-1) are linarized with the index l. Where l is a usual row-major linear mapping index.

\begin{equation} l(i, r) = i \cdot R + r
\end{equation} 

\begin{equation} M_l = M_{i, r}
\end{equation} 

\begin{equation} i_l = [l / R]
\end{equation} 

\begin{equation} r_l = l  \%  R
\end{equation} 

\begin{equation} a_l = 4 \cdot 2^{-r_l} \sum_{k} M_{ki_l} \{ 2^{-r_l} M_{ki_l} - (y_k + \sum_{j} M_{kj}) \}
\end{equation} 



In [5]:
def get_mat_weights(m,y,r):
    d = np.size(y)
    dm1, dm2 = np.shape(m)
    if dm1!=dm2:
        print("Error: m is not a symmetric matrix!")
        exit()
    if d!=dm1:
        print("Error: y does not have the same dimensions as m!")
        exit()
    lsize = d*r
    al = np.zeros(lsize)    
    for l in range(lsize):
        rl = int(np.mod(l,r))
        fl = (0.5)**(rl)
        #print 'fl = ', fl
        asum = 0.0
        il = int(np.floor_divide(l,r))
        for k in np.arange(d):
            nm = 2.*m[k,il]
            msum = 0.0
            for j in np.arange(d):
                # if j <= k:
                msum += m[k,j]
            asum += nm*(0.5)**(rl)*(nm*(0.5)**(rl) - 2.*(y[k]+msum))
        al[l] = asum
    return al  

\begin{equation} b_{lm} = 4 \cdot 2^{-(r_l+r_{l'})} \sum_{k} M_{ki_l} M_{ki_m} 
\end{equation} 

In [6]:
def get_mat_strengths(m,r):
    dm1, dm2 = np.shape(m)
    if dm1!=dm2:
        print("Error: m is not a symmetric matrix!")
        exit()
    d = dm1
    lsize = d*r
    bij = np.zeros([lsize,lsize]) # array form of strengths
    for i in range(lsize):
        for j in range(lsize):
            pij = (np.mod(i,r)+np.mod(j,r))
            fl = (0.5)**pij        
            il = np.floor_divide(i,r)
            jl = np.floor_divide(j,r)
            msum = 0.0
            for k in np.arange(d):            
                if j!=i:
                    msum += m[k,il]*m[k,jl]
            bij[i,j] = 4.0*fl*msum
    return bij

In [7]:
def normalize_qubo(a,b):
    na = np.max(np.abs(a))
    nb = np.max(np.abs(b))
    w = min(2.0/na, 1.0/nb)
    an = w*a
    bn = w*b
    return an, bn, w

In [8]:
def normalize_matrix_equation(M,Y):
    # Compute norms of M, Y and X.
    mnorm = np.linalg.norm(M,-2) # "negative 2"-norm of M
    ynorm = np.linalg.norm(Y)    # ordinary 2-norm of Y
    xnorm = ynorm/mnorm
    # Normalize M and Y.
    m = M/mnorm
    y = Y/ynorm
    return m, y, xnorm

Compute solution x from sampleset qubits

In [9]:
def compute_x_from_q(q_solution, r, n, i):
    # Implementation should be fine
    # Compute the coefficients of x from solved q: binary -> floating
    x = 0
    for j in range(n*r):
        i_l = j // r
        r_l = j % r
        if i == i_l :
            x += 2**(-r_l) * q_solution[j]
    x = 2 * x - 1
    return x

## Let's imagine a matrix

In [10]:
# Initialize matrix
# Test 1 (a):
M = np.array([[1/2, 3/2], [3/2, 1/2]])
#M = np.array([[2, -1], [-1/2, 1/2]])
y = np.array([1, 0])
# Initialize solution
#y = np.array([1, 0])
M, y, x_norm = normalize_matrix_equation(M, y)
# max_v = max(np.max(M), np.max(y))
# min_v = min(np.min(M), np.min(y))
# M = 2 * ((M-min_v)/(max_v-min_v)) - 1
# y = 2 * ((y-min_v)/(max_v-min_v)) - 1
print(M)
print(y)
# Matrix size
n = M.shape[0]
# Binary bit size
r = 4
# 1-D range
total_l = n*r

[[0.5 1.5]
 [1.5 0.5]]
[1. 0.]


## Construct QUBO problem as BQM for a DWave solver
To implement a usable problem for the DWave solver, we need to construct a matrix Q for our QUBO model. Q contains it's linear coefficient along the diagonal and the quadratic coefficients 

In [20]:
def construct_qubo_dict(M, y, binary_range):
    al = get_mat_weights(M, y, binary_range)
    bij = get_mat_strengths(M, binary_range)
    al, bij, w = normalize_qubo(al, bij)
    l = M.shape[0] * binary_range
    Q = {}
    for i in range(l):
        Q[(i, i)] = al[i]
        for j in range(i+1, l):
            Q[(i, j)] = bij[i, j]
    return Q

In [21]:
Q = construct_qubo_dict(M, y, binary_range)
print(Q)

{(0, 0): -1.3333333333333333, (0, 1): 0.8333333333333333, (0, 2): 0.41666666666666663, (0, 3): 0.20833333333333331, (0, 4): 1.0, (0, 5): 0.5, (0, 6): 0.25, (0, 7): 0.125, (1, 1): -1.0833333333333333, (1, 2): 0.20833333333333331, (1, 3): 0.10416666666666666, (1, 4): 0.5, (1, 5): 0.25, (1, 6): 0.125, (1, 7): 0.0625, (2, 2): -0.6458333333333333, (2, 3): 0.05208333333333333, (2, 4): 0.25, (2, 5): 0.125, (2, 6): 0.0625, (2, 7): 0.03125, (3, 3): -0.3489583333333333, (3, 4): 0.125, (3, 5): 0.0625, (3, 6): 0.03125, (3, 7): 0.015625, (4, 4): -2.0, (4, 5): 0.8333333333333333, (4, 6): 0.41666666666666663, (4, 7): 0.20833333333333331, (5, 5): -1.4166666666666665, (5, 6): 0.20833333333333331, (5, 7): 0.10416666666666666, (6, 6): -0.8125, (6, 7): 0.05208333333333333, (7, 7): -0.43229166666666663}


In [40]:
bqm = dimod.BQM.from_qubo(Q)
sampleset = EmbeddingComposite(DWaveSampler()).sample(bqm, num_reads=100, label="QUBO Matrix Inversion")
print(sampleset)

    0  1  2  3  4  5  6  7    energy num_oc. chain_.
0   0  1  1  1  0  1  1  1 -3.244792      21     0.0
1   0  1  1  0  0  1  1  1 -3.161458      14     0.0
3   0  1  1  1  1  0  1  1 -3.078125       6     0.0
2   0  0  1  1  1  1  1  1 -3.078125       3     0.0
4   0  1  1  0  1  0  1  1 -3.057292       5     0.0
5   0  0  1  1  1  1  1  0 -3.057292       8     0.0
6   0  1  0  1  1  0  1  1 -3.036458       3     0.0
7   0  0  1  1  1  1  0  1 -3.036458       3     0.0
8   0  1  0  1  1  1  0  1 -3.015625       1     0.0
9   0  0  1  0  1  1  1  1 -3.015625       3     0.0
11  0  1  1  0  1  1  0  1 -3.005208       4     0.0
10  0  1  0  1  1  1  1  0 -3.005208       1     0.0
12  0  1  1  1  1  1  0  1 -2.994792       1     0.0
13  0  1  0  0  1  1  1  1 -2.984375       1     0.0
15  0  1  1  0  1  1  0  0 -2.979167       1     0.0
16  0  1  1  0  1  1  1  0 -2.979167       3     0.0
14  0  1  1  0  1  0  1  0 -2.979167       1     0.0
17  0  1  0  0  1  1  1  0 -2.979167       1  

In [23]:
first_solution = np.array(list(sampleset.first.sample.items()))[:, 1]
#soln = map(int,(np.array(first_solution)+1)/2)
#print(soln)

In [15]:
def qa2xa(q,r):    
    if np.size(q)%r!=0:
        print("Error: Size of Boolean array, q, must be divisible by r!")
        quit()
    else:
        d = np.size(q)//r
        chi = np.zeros(d)
        x = np.zeros(d)
        for i in range(d):
            lsum = 0.0
            for l in range(d*r):
                il = np.floor_divide(l,r)
                if i==il:
                    rl = np.mod(l,r)
                    lsum += (2.**(-rl))*q[l]
                else:
                    lsum += 0.0
            chi[i] = lsum
            x[i] = 2.*chi[i]-1.0
        return x

In [19]:
dec = qa2xa(first_solution, binary_range)
print(dec)

[0.75 0.75]


In [17]:
x = np.zeros((n))
# Feed through each component of x
for i in range(n):
    x[i] = compute_x_from_q(first_solution, r, n, i)
print(x)

NameError: name 'soln' is not defined

In [ ]:
classical_result = np.linalg.inv(M).dot(y)
print(classical_result)

[-0.25  0.75]


# Run matrix inversion on quantum annealer without intermediate steps

This method computes the solution of the linear equations using the annealing method, described above, without any intermediate steps. Just pass on the matrix, vector and binary range.

In [ ]:
def matrix_inversion_dwave(M, y, binary_range=4):
    y_norm = np.inner(y,y)
    M = 2*M/y_norm -1
    y = 2*y/y_norm -1
    Q = construct_qubo_dict(M, y, binary_range)
    # Sampling solutions for 1000 reads
    offset = np.inner(y, y)
    # minimum = min(np.min(M), np.min(y))
    # maximum = max(np.max(M), np.max(y))
    # M = 2 * ((M-minimum)/(maximum-minimum)) - 1
    # y = 2 * ((y-minimum)/(maximum-minimum)) - 1
    # offset = 0
    #print("Offset: ", offset)
    #bqm = dimod.BinaryQuadraticModel(L, Q, offset, dimod.Vartype.BINARY)
    bqm = dimod.BinaryQuadraticModel.from_qubo(Q, offset=offset)
    sampleset = EmbeddingComposite(DWaveSampler()).sample(bqm, num_reads=1000, label="QUBO Matrix Inversion")
    first_solution = np.array(list(sampleset.first.sample.items()))[:, 1]
    x = np.zeros((n))
    # Feed through each component of x
    for i in range(n):
        x[i] = compute_x_from_q(first_solution, binary_range, n, i)
    return x #, sampleset

# 2 x 2 test cases from the paper

In [ ]:
# Initialize matrix
# Test 1 (a):
M_a = np.array([[1/2, 3/2], [3/2, 1/2]])
y_a = np.array([1, 0])
# Test 1 (b):
M_b = np.array([[1/2, 3/2], [3/2, 1/2]])
y_b = np.array([0, 1])
# Test 1 (c):
M_c = np.array([[2, -1], [-1/2, 1/2]])
y_c = np.array([1, 0])
# Test 1 (d):
M_d = np.array([[1, 2], [1/2, 1/2]])
y_d = np.array([1, 0])
# Test 1 (e):
M_e = np.array([[3, 2], [2, 1]])
y_e = np.array([1, 1])
# Test 1 (f):
M_f = np.array([[3, 2], [2, 1]])
y_f = np.array([1, 1])
# Test 1 (g):
M_g = np.array([[0, -2], [-2, -3/3]])
y_g = np.array([1, 1/4])
# Test 1 (h):
M_h = np.array([[0, -2], [-2, -3/3]])
y_h = np.array([-0.5, -0.875])
# Test 1 (i): Ill-conditioned
M = np.array([[1, 2], [2, 3.999]])
y = np.array([4.0, 7.999])
# Test 1 (i): Pre-conditioned version of test 1 (i)
M_i = np.array([[1.80026, 1.6019], [1.6019, 4.19974]])
y_i = np.array([5.2007, 7.40013])
# Matrix size
n = M.shape[0]
# Binary bit size
r = 4
# 1-D range
total_l = n*r
Ms = [M_a, M_b, M_c, M_d, M_e, M_f, M_g, M_h, M_i]
ys = [y_a, y_b, y_c, y_d, y_e, y_f, y_g, y_h, y_i]

# 3 x 3 test cases from the paper

In [ ]:
# # # Initialize matrix
# # Test 1 (a):
# M_a = np.array([[0.0, -2.0, 0.0], [-2.0, 1.5, 0.0], [0.0, 0.0, 1.0]])
# y_a = np.array([1.0, 0.25, 1.0])
# # # Test 1 (b):
# M_b = np.array([[0.0, -2.0, 0.0], [-2.0, 1.5, 0.0], [0.0, 0.0, 1.0]])
# y_b = np.array([1.0, 0.25, 0.0])
# # # Test 1 (c):
# M_c = np.array([[1.0, 0.0, 0.0], [0.0, 0.0, -2.0], [0.0, -2.0, -1.5]])
# y_c = np.array([1.0, 0.0, 0.25])
# # # Test 1 (d):
# M_d = np.array([[1.0, 0.0, 0.0], [0.0, 0.0, -2.0], [0.0, -2.0, -1.5]])
# y_d = np.array([1.0, 1.0, 0.25])
# # # Test 1 (e):
# M_e = np.array([[0.0, -2.0, 0.0], [-2.0, 1.5, 0.0], [0.0, 0.0, 1.0]])
# y_e = np.array([0.0, 1.0, 0.25])
# # # Test 1 (f):
# M_f = np.array([[-4.0, 6.0, 1.0], [8.0, -11.0, -2.0], [-3.0, 4.0, 1.0]])
# y_f = np.array([0.75, -1.25, 0.25])
# # # Test 1 (g):
# M_g = np.array([[6.1795, 11.8207, 2.0583], [15.673, -7.56717, -3.8520], [-5.6457, 7.96872, 15.9418]])
# y_g = np.array([1.4114, 0.9972, 9.9643])

# # Matrix size
# n = M.shape[0]
# # Binary bit size
# r = 4
# # 1-D range
# total_l = n*r

# Ms = [M_a, M_b, M_c, M_d, M_e, M_f, M_g, M_h, M_i]
# ys = [y_a, y_b, y_c, y_d, y_e, y_f, y_g, y_h, y_i]

Compute for on test case

In [ ]:
result = matrix_inversion_dwave(M_b, y_b, binary_range=4)
print(result)

Compute for all test cases

In [ ]:
for i in range(len(Ms)):
    annealing_result = matrix_inversion_dwave(Ms[i], ys[i], binary_range=4)
    classical_result = np.linalg.inv(Ms[i]).dot(ys[i])
    print("Annealing: ", annealing_result, "Classical: ", classical_result)
    

| Annealing result | Classical result | 
| --- | --- | 
| [-0.25 0.75] | [-0.25 0.75] | 
| [ 0.75 -0.25] | [ 0.75 -0.25] |
| [-0.25 -1.  ] |  [1. 1.] |
|  [0.75 0.75] |  [-1.  1.] |
|  [0.75 0.75] |  [ 1. -1.] |
|  [0.75 0.75] |  [ 1. -1.] |
|  [-0.25 -0.25] |  [ 0.125 -0.5  ] |
|  [0.75 0.75] |  [0.3125 0.25  ] |
|  [2.75 0.75] |  [1.9996474  0.99932254] |